# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), January 30, 2019**

Let's get started by loading the `DataFrames` package.

In [1]:
using DataFrames, Random

## Constructors and conversion

### Constructors

In this section, you'll see many ways to create a `DataFrame` using the `DataFrame()` constructor.

First, we could create an empty DataFrame,

In [2]:
DataFrame() # empty DataFrame

""
""


Or we could call the constructor using keyword arguments to add columns to the `DataFrame`.

In [3]:
DataFrame(A=1:3, B=rand(3), C=randstring.([3,3,3]))

,A,B,C
,Int64,Float64,String
1,1,0.697324,NwZ
2,2,0.718295,6TW
3,3,0.258834,YVL


We can create a `DataFrame` from a dictionary, in which case keys from the dictionary will be sorted to create the `DataFrame` columns.

In [4]:
x = Dict("A" => [1,2], "B" => [true, false], "C" => ['a', 'b'])
DataFrame(x)

,A,B,C
,Int64,Bool,Char
1,1,true,'a'
2,2,false,'b'


Rather than explicitly creating a dictionary first, as above, we could pass `DataFrame` arguments with the syntax of dictionary key-value pairs. 

Note that in this case, we use symbols to denote the column names and arguments are not sorted. For example, `:A`, the symbol, produces `A`, the name of the first column here:

In [5]:
DataFrame(:A => [1,2], :B => [true, false], :C => ['a', 'b'])

,A,B,C
,Int64,Bool,Char
1,1,true,'a'
2,2,false,'b'


Here we create a `DataFrame` from a vector of vectors, and each vector becomes a column.

In [6]:
DataFrame([rand(3) for i in 1:3])

,x1,x2,x3
,Float64,Float64,Float64
1,0.811443,0.232797,0.759062
2,0.419609,0.420083,0.044275
3,0.824869,0.43918,0.927567


It is not allowed to pass a vector of scalars to `DataFrame` constructor.

In [7]:
DataFrame(rand(3))

ArgumentError: ArgumentError: unable to construct DataFrame from Array{Float64,1}

Instead use a transposed vector if you have a vector of atoms (in this way you effectively pass a two dimensional array to the constructor which is supported).

In [8]:
DataFrame(permutedims([1, 2, 3]))

,x1,x2,x3
,Int64,Int64,Int64
1,1,2,3


or pass a vector of `NamedTuple`s

In [9]:
v = [(a=1, b=2), (a=3, b=4)]
DataFrame(v)

,a,b
,Int64,Int64
1,1,2
2,3,4


Pass a second argument to give the columns names in case you pass a vector of vectors.

In [10]:
DataFrame([1:3, 4:6, 7:9], [:A, :B, :C])

,A,B,C
,Int64,Int64,Int64
1,1,4,7
2,2,5,8
3,3,6,9


Alternatively you can pass a `NamedTupe` of vectors:

In [11]:
n = (a=1:3, b=11:13)
DataFrame(n)

,a,b
,Int64,Int64
1,1,11
2,2,12
3,3,13


Here we create a `DataFrame` from a matrix,

In [12]:
DataFrame(rand(3,4))

,x1,x2,x3,x4
,Float64,Float64,Float64,Float64
1,0.82056,0.0673816,0.654091,0.674425
2,0.415571,0.291093,0.206933,0.137289
3,0.545475,0.164674,0.452445,0.262285


and here we do the same but also pass column names.

In [13]:
DataFrame(rand(3,4), Symbol.('a':'d'))

,a,b,c,d
,Float64,Float64,Float64,Float64
1,0.412906,0.522153,0.399317,0.824805
2,0.383185,0.517894,0.84653,0.511331
3,0.425717,0.870048,0.797669,0.624747


We can also construct an uninitialized DataFrame.

Here we pass column types, names and number of rows; we get `missing` in column :C because `Any >: Missing`.

In [14]:
DataFrame([Int, Float64, Any], [:A, :B, :C], 1)

,A,B,C
,Int64,Float64,Any
1,148706496,7.35043e-316,missing


Here we create a `DataFrame` where `:C` is `#undef`

In [15]:
DataFrame([Int, Float64, String], [:A, :B, :C], 1)

,A,B,C
,Int64,Float64,String
1,115551456,6.08127e-316,#undef


To initialize a `DataFrame` with column names, but no rows use

In [16]:
DataFrame([Int, Float64, String], [:A, :B, :C], 0) 

,A,B,C
,Int64,Float64,String


or

In [17]:
DataFrame(A=Int[], B=Float64[], C=String[])

,A,B,C
,Int64,Float64,String


This syntax gives us a quick way to create an uninitialized homogenous `DataFrame`.

In [18]:
DataFrame(Int, 3, 5)

,x1,x2,x3,x4,x5
,Int64,Int64,Int64,Int64,Int64
1,21671613376,120950032,21731381280,21731410272,21728452544
2,21689211440,21689211440,81529040,148706944,148737520
3,81528816,81528816,148706304,21670183040,148707072


This example is similar, but has nonhomogenous columns.

In [19]:
DataFrame([Int, Float64], 4)

,x1,x2
,Int64,Float64
1,281479271677952,1.39067e-309
2,0,0.0
3,0,0.0
4,0,0.0


Finally, we can create a `DataFrame` by copying an existing `DataFrame`.

Note that `copy` creates a shallow copy.

In [20]:
x = DataFrame(a=1:2, b='a':'b')
y = copy(x)
(x === y), isequal(x, y)

(false, true)

Calling `DataFrame` on a `DataFrame` object also performs a shallow copy.

In [21]:
x = DataFrame(a=1:2, b='a':'b')
y = DataFrame(x)
(x === y), isequal(x, y)

(false, true)

You can create a similar uninitialized `DataFrame` based on an original one:

In [22]:
similar(x)

,a,b
,Int64,Char
1,148705152,'\x04\xdc\x0b\x70'
2,148706176,'\0'


In [23]:
similar(x, 0) # number of rows in a new DataFrame passed as a second argument

,a,b
,Int64,Char


You can also create a new `DataFrame` from `SubDataFrame` or `DataFrameRow` (discussed in detail later in the tutorial)

In [24]:
sdf = view(x, [1,1], :)

,a,b
,Int64,Char
1,1,'a'
2,1,'a'


In [25]:
typeof(sdf)

SubDataFrame{DataFrame,DataFrames.Index,Array{Int64,1}}

In [26]:
DataFrame(sdf)

,a,b
,Int64,Char
1,1,'a'
2,1,'a'


In [27]:
dfr = x[1, :]

,a,b
,Int64,Char
1,1,'a'


In [28]:
DataFrame(dfr)

,a,b
,Int64,Char
1,1,'a'


### Conversion to a matrix

Let's start by creating a `DataFrame` with two rows and two columns.

In [29]:
x = DataFrame(x=1:2, y=["A", "B"])

,x,y
,Int64,String
1,1,A
2,2,B


We can create a matrix by passing this `DataFrame` to `Matrix`.

In [30]:
Matrix(x)

2×2 Array{Any,2}:
 1  "A"
 2  "B"

This would work even if the `DataFrame` had some `missing`s:

In [31]:
x = DataFrame(x=1:2, y=[missing,"B"])

,x,y
,Int64,String⍰
1,1,missing
2,2,B


In [32]:
Matrix(x)

2×2 Array{Any,2}:
 1  missing
 2  "B"    

In the two previous matrix examples, Julia created matrices with elements of type `Any`. We can see more clearly that the type of matrix is inferred when we pass, for example, a `DataFrame` of integers to `Matrix`, creating a 2D `Array` of `Int64`s:

In [33]:
x = DataFrame(x=1:2, y=3:4)

,x,y
,Int64,Int64
1,1,3
2,2,4


In [34]:
Matrix(x)

2×2 Array{Int64,2}:
 1  3
 2  4

In this next example, Julia correctly identifies that `Union` is needed to express the type of the resulting `Matrix` (which contains `missing`s).

In [35]:
x = DataFrame(x=1:2, y=[missing,4])

,x,y
,Int64,Int64⍰
1,1,missing
2,2,4


In [36]:
Matrix(x)

2×2 Array{Union{Missing, Int64},2}:
 1   missing
 2  4       

Note that we can't force a conversion of `missing` values to `Int`s!

In [37]:
Matrix{Int}(x)

ErrorException: cannot convert a DataFrame containing missing values to Matrix{Int64} (found for column y)

### Conversion to `NamedTuple` related tabular structures

In [38]:
x = DataFrame(x=1:2, y=["A", "B"])

,x,y
,Int64,String
1,1,A
2,2,B


In [39]:
using Tables

First we convert a `DataFrame` into a `NamedTuple` of vectors

In [40]:
ct = Tables.columntable(x)

(x = [1, 2], y = ["A", "B"])

Next we convert it into a vector of `NamedTuples`

In [41]:
rt = Tables.rowtable(x)

2-element Array{NamedTuple{(:x, :y),Tuple{Int64,String}},1}:
 (x = 1, y = "A")
 (x = 2, y = "B")

We can perform the conversions back to a `DataFrame` using a standard constructor call:

In [42]:
DataFrame(ct)

,x,y
,Int64,String
1,1,A
2,2,B


In [43]:
DataFrame(rt)

,x,y
,Int64,String
1,1,A
2,2,B


### Handling of duplicate column names

We can pass the `makeunique` keyword argument to allow passing duplicate names (they get deduplicated)

In [44]:
df = DataFrame(:a=>1, :a=>2, :a_1=>3; makeunique=true)

,a,a_2,a_1
,Int64,Int64,Int64
1,1,2,3


Otherwise, duplicates are not allowed.

In [45]:
df = DataFrame(:a=>1, :a=>2, :a_1=>3)

ArgumentError: ArgumentError: Duplicate variable names: :a. Pass makeunique=true to make them unique using a suffix automatically.

Finallly observe that `nothing` is not printed when displaying a `DataFrame`:

In [46]:
DataFrame(x=[1, nothing], y=[nothing, "a"])

,x,y
,Union…,Union…
1,1,
2,,a
